In [136]:
from astropy.io import fits
from astropy.table import Table
from astropy.nddata import NDDataArray, StdDevUncertainty
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools
%matplotlib inline

# Basic Steps
For a given spatial pixel $p = (x,y)$ with $N$ values of observed line ratios $R_N(p)$ with errors on the line ratios $E_N(p)$ in same units as $R_N$, we use predicted line ratios as function of $A_N(G_0,n)$, to compute differences $\delta_N$ across all $A$:
$${\delta_N^2(p,G_0,n)} = {\left[ R_N(p) - A_N(G_0,n)\over E_N(x)\right] }^2$$
 
The reduced chi-squared for spatial pixel $i$ is then given by

$$\chi^2(p,G_0,n) =  \sum_N {\delta_N^2(p,G_0,n) \over N-1}$$

and the best-fit value of $(G_0,n)$ is the minimum of $\chi^2$  The range of $(G_0,n)$ over which $A_N$ have been computed must be large enough to encompass physically reasonable values in the ISM, typically $10^{-1} < G_0 < 10^7$ in Habing units and $1 < n < 10^7$ cm$^{-3}$

For a spatial map, we do the above computation over all $p$ to produce a spatial maps of $\chi^2$ and best-fit spatial maps of $(G_0,n)$.



In [145]:
# maybe this is just better storing this info in a Table.  Then adding models is 
# just adding to the table
class ModelRatioData:
    def __init__(self,numerator, denominator, identifier,filename,metallicity=1):
        self._numerator = numerator
        self._denominator =denominator
        self._identifier = identifier
        self._filename = filename + "web"
        self._metallicity= metallicity
        
    @property
    def identifier(self): return self._identifier
   
    @property 
    def filename(self): return self._filename
    
    @property 
    def metallicity(self): return self._metallicity
    
    @property
    def isSolarMetallicity(self):
        return self._metallicity == 1

class Measurement(NDDataArray):
    def __init__(self,identifier,flux,error):
        NDDataArray.__init__(self,data=flux,uncertainty=StdDevUncertainty(error))
        self._identifier = identifier
        self._flux      = flux
        self._error     = error
        
    @property
    def flux(self):
        return self._flux
    @property
    def id(self):
        return self._identifier
    @property
    def error(self):
        return self._error
    
    def __add__(self,other):
        pass
    def __sub__(self,other):
        pass
    def __mul__(self,other):
        pass
    def __truediv__(self,other):
        pass
    
class PDRutils:
    def __init__(self,models):
        if type(models) == str:
            self.initializeFromFile(models)
        else:
            self._models = models
        
    def initializeFromFile(self,filename):
        self._models=Table.read(filename,format="ascii.ipac")
        self._models.add_index("label")
        
    def find_pairs(self,m):
        for q in itertools.product(m,m):
            s = q[0]+"/"+q[1]
            if s in self._models["label"]:
                yield(s)
    
    def find_files(self,m,ext="fits"):
        for q in itertools.product(m,m):
            s = q[0]+"/"+q[1]
            if s in self._models["label"]:
                yield(self._models.loc[s]["filename"]+"."+ext)
                
    def read_fits(self,m):
        d = "/n/lupus/mpound/WITS/Docs/pdrt/"
        self._fitsfiles = dict()
        for p in self.find_files(m):
            self._fitsfiles[p] = fits.open(d+p)
    
                

In [116]:
ratiodict = {
    "OI_145/OI_63"   : "oioi",
    "OI_145/CII_158" : "o145cii",
    "OI_63/CII_158"  : "oicp",
    "CII_158/CI_609" : "ciici609",
    "CI_370/CI_609"  : "cici",
    "CII_158/CO_10"  : "ciico",
    "CI_609/CO_10"   : "cico",
    "CI_609/CO_21"   : "cico21",
    "CI_609/CO_32"   : "cico32",
    "CI_609/CO_43"   : "cico43",
    "CI_609/CO_54"   : "cico54",
    "CI_609/CO_65"   : "cico65",
    "CO_21/CO_10"    : "co2110",
    "CO_32/CO_10"    : "co3210",
    "CO_32/CO_21"    : "co3221",
    "CO_43/CO_21"    : "co4321",
    "CO_65/CO_10"    : "co6510",
    "CO_65/CO_21"    : "co6521",
    "CO_65/CO_54"    : "co6554",
    "CO_76/CO_10"    : "co7610",
    "CO_76/CO_21"    : "co7621",
    "CO_76/CO_43"    : "co7643",
    "CO_76/CO_54"    : "co7654",
    "CO_76/CO_65"    : "co7665",
    "CO_87/CO_54"   : "co8754",
    "CO_87/CO_65"   : "co8765",
    "CO_98/CO_54"   : "co9854",
    "CO_98/CO_65"   : "co9865",
    "CO_109/CO_54"   : "co10954",
    "CO_109/CO_65"   : "co10965",
    "CO_1110/CO_54"   : "co111054",
    "CO_1110/CO_65"   : "co111065",
    "CO_1211/CO_54"   : "co121154",
    "CO_1211/CO_65"   : "co121165",
    "CO_1312/CO_54"   : "co131254",
    "CO_1312/CO_65"   : "co131265",
    "CO_1413/CO_54"   : "co141354",
    "CO_1413/CO_65"   : "co141365",
    "OI_63+CII_158/FIR"     : "fir",
    "OI_145+CII_158/FIR"  : "firoi145",
  "SIII_Z1/FEII_Z1"  : "siii35feii26z1",
    "SIII_Z3/FEII_Z3"  : "siii35feii26z3",
    "H200S1_Z1/H200S0_Z1" : "h200s1s0z1",
    "H200S1_Z3/H200S0_Z3" : "h200s1s0z3",
    "H200S2_Z1/H200S0_Z1" : "h200s2s0z1",
    "H200S2_Z3/H200S0_Z3" : "h200s2s0z3",
    "H200S2_Z1/H200S1_Z1" : "h200s2s1z1",
    "H200S2_Z3/H200S1_Z3" : "h200s2s1z3",
    "H200S3_Z1/H200S1_Z1" : "h200s3s1z1",
    "H200S3_Z3/H200S1_Z3" : "h200s3s1z3",
    "H200S1_Z1/SIII_Z1" : "h200s1siiiz1",
    "H200S1_Z3/SIII_Z3" : "h200s1siiiz3",
    "H200S2_Z1/SIII_Z1" : "h200s2siiiz1",
    "H200S2_Z3/SIII_Z3" : "h200s2siiiz3",
    "H264Q1_Z1/H210S1_Z1" : "h264q110s1z1",
    "H264Q1_Z3/H210S1_Z3" : "h264q110s1z3"
}
a = list()
b = list()
for r in ratiodict:
    nd = r.split("/")
    if ("Z3" in r):
        z=3
    else:
        z=1
    b.append((nd[0],nd[1],r,ratiodict[r]+"web",z))
    a.append(ModelRatioData(nd[0],nd[1],r,ratiodict[r],z))
t = Table(rows=b,names=("numerator","denominator","label","filename","z"))
t.add_index("label")
t.write("current_models.tab",format="ascii.ipac",overwrite=True)

df = t.to_pandas()

In [146]:
m1 = Measurement("OI_145",30.,5.)
m2 = Measurement("CI_370",10.,10.)
m3 = Measurement("CO_10",500.,50.)
m4 = Measurement("CII_158",100.,10.)
m = [m1.id,m2.id,m3.id,m4.id]

In [131]:
p = PDRutils("current_models.tab")
for z in p.find_files(m):
    print(z)

o145ciiweb.fits
ciicoweb.fits


In [132]:
p.read_fits(m)
print(p._fitsfiles)

{'o145ciiweb.fits': [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7f1108794a90>], 'ciicoweb.fits': [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7f110877e780>]}


In [134]:
p._fitsfiles['ciicoweb.fits'][0].data

array([[3.103900e+02, 3.258060e+02, 2.910660e+02, 2.168180e+02,
        1.381730e+02, 8.013700e+01, 4.349590e+01, 2.313250e+01,
        1.213950e+01, 6.615970e+00, 3.791950e+00, 1.988800e+00,
        1.172910e+00, 5.801750e-01, 2.094490e-01, 6.945240e-02,
        1.045710e-02, 1.346630e-03, 1.928080e-04, 4.444440e-05,
        1.457690e-05, 5.465120e-06, 2.297710e-06, 7.799230e-07,
        2.698800e-07],
       [4.950490e+02, 5.422080e+02, 5.114940e+02, 4.004470e+02,
        2.734500e+02, 1.704920e+02, 1.030769e+02, 6.179780e+01,
        3.717300e+01, 2.349480e+01, 1.430560e+01, 9.812330e+00,
        5.669100e+00, 2.964710e+00, 1.432990e+00, 4.532710e-01,
        1.050380e-01, 1.504950e-02, 1.953300e-03, 3.353470e-04,
        8.974360e-05, 2.930920e-05, 1.046050e-05, 3.453040e-06,
        1.232100e-06],
       [7.483220e+02, 8.496730e+02, 8.395420e+02, 6.960350e+02,
        4.992220e+02, 3.315620e+02, 2.147060e+02, 1.380950e+02,
        9.087300e+01, 6.049380e+01, 4.255870e+01, 2.892940

In [147]:
m1+m2

TypeError: __add__() takes 1 positional argument but 2 were given